In [2]:
import pandas as pd
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [ ]:
#len(train)

1200000

In [ ]:
#train.isnull().sum()

id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [5]:
import numpy as np

# Number of Dependents
train['Number of Dependents'] = train['Number of Dependents'].astype('Int64')
train['Number of Dependents'] = train['Number of Dependents'].astype('category')
train['Number of Dependents'] = train['Number of Dependents'].cat.add_categories('Missing')
train['Number of Dependents'] = train['Number of Dependents'].fillna('Missing')

# Age
train['Age'] = train['Age'].fillna(train['Age'].median())

# Annual Income
train['Annual Income'] = train['Annual Income'].fillna(train['Annual Income'].median())

# Health Score
train['Health Score'] = train['Health Score'].fillna(train['Health Score'].median())

# Credit Score
train['Credit Score'] = train['Credit Score'].fillna('Missing')

# Customer Feedback
train['Customer Feedback'] = train['Customer Feedback'].fillna('No Feedback')

# Marital Status
train['Marital Status'] = train['Marital Status'].fillna('Unknown')

# Vehicle Age 결측 행 삭제
train = train[train['Vehicle Age'].notnull()]

# Insurance Duration 결측 행 삭제
train = train[train['Insurance Duration'].notnull()]

In [ ]:
#train.isnull().sum()

id                           0
Age                          0
Gender                       0
Annual Income                0
Marital Status               0
Number of Dependents         0
Education Level              0
Occupation              358074
Health Score                 0
Location                     0
Policy Type                  0
Previous Claims         364028
Vehicle Age                  0
Credit Score                 0
Insurance Duration           0
Policy Start Date            0
Customer Feedback            0
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64

In [7]:
train1 = train.copy()
train1['Occupation'] = train1['Occupation'].fillna('Unknown')

train1['Previous Claims'] = train1['Previous Claims'].fillna(0)

In [8]:
train2 = train.copy()
train2['Occupation'] = train2['Occupation'].fillna('Unknown')

train2['Previous Claims'] = train2['Previous Claims'].astype('Int64')
train2['Previous Claims'] = train2['Previous Claims'].astype('category')
train2['Previous Claims'] = train2['Previous Claims'].cat.add_categories('Missing')
train2['Previous Claims'] = train2['Previous Claims'].fillna('Missing')

In [9]:
train3 = train.copy()
occupation_income_mean = train.groupby('Occupation')['Annual Income'].mean()
def fill_occupation(row):
   if pd.isnull(row['Occupation']):
       diffs = (occupation_income_mean - row['Annual Income']).abs()
       return diffs.idxmin()
   else:
       return row['Occupation']

train3['Occupation'] = train3.apply(fill_occupation, axis=1)

train3['Previous Claims'] = train3['Previous Claims'].fillna(0)

In [10]:
train4 = train.copy()
occupation_income_mean = train.groupby('Occupation')['Annual Income'].mean()
def fill_occupation(row):
   if pd.isnull(row['Occupation']):
       diffs = (occupation_income_mean - row['Annual Income']).abs()
       return diffs.idxmin()
   else:
       return row['Occupation']

train4['Occupation'] = train4.apply(fill_occupation, axis=1)

train4['Previous Claims'] = train4['Previous Claims'].astype('Int64')
train4['Previous Claims'] = train4['Previous Claims'].astype('category')
train4['Previous Claims'] = train4['Previous Claims'].cat.add_categories('Missing')
train4['Previous Claims'] = train4['Previous Claims'].fillna('Missing')

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


# 2. 타겟 설정
target = 'Premium Amount'
y1 = train1[target]
X1 = train1.drop(columns=[target])


# 3. 결측치 처리 및 범주형 컬럼 지정
cat_features = X1.select_dtypes(include=['object', 'category']).columns.tolist()


# NaN을 'Missing'으로 문자열 처리 (CatBoost 호환)
for col in cat_features:
    X1[col] = X1[col].astype(str).fillna("Missing")


# 숫자형 결측치 처리 (중앙값 대체)
for col in X1.select_dtypes(include=[np.number]).columns:
    if X1[col].isnull().sum() > 0:
        X1[col] = X1[col].fillna(X1[col].median())


# 4. 학습/검증 데이터 분리
X_train1, X_val1, y_train1, y_val1 = train_test_split(X1, y1, test_size=0.2, random_state=42)


# 5. 모델 정의
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    eval_metric='MAE',
    cat_features=cat_features,
    random_seed=42,
    verbose=100
)


# 6. 모델 학습
model.fit(X_train1, y_train1, eval_set=(X_val1, y_val1), use_best_model=True)


# 7. 예측 및 평가
y_pred1 = model.predict(X_val1)
print(f":pushpin: MAE: {mean_absolute_error(y_val1, y_pred1):.4f}")
